In [1]:
%reload_ext autoreload
%autoreload 2

### TODO: Make into special script

In [2]:
import h5py
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
import numpy as np
from io import BytesIO
import base64

In [3]:
in_f = "../data/output/data_100.hdf5"
out_f = "../data/output/data_100.json"

In [4]:
def _image_to_string(array):
    """ Converts numpy array to base64 string. """
    pil_array = Image.fromarray((array * 225).astype(np.uint8))
    buff = BytesIO()
    pil_array.save(buff, format="JPEG")
    array_string = base64.b64encode(buff.getvalue()).decode("utf-8")
    return array_string

def to_json_data(in_f, out_f):
    f = h5py.File(in_f, 'r')
    rows = []
    for i, fname in enumerate(f['images']):
        img = f['images'][fname]
        img_data = np.asarray(img['image']).transpose(1,2,0)
        img_str = _image_to_string(img_data)
        row = {
            "fname": fname,
            "bbox": list(img['bbox_polygons'][()]),
            "image": img_str,
            "saliency": list(img['saliency_polygons'][()]),
            "label": img.attrs["label"],
            "prediction": img.attrs['prediction'],
            "bbox_proportion_score": img.attrs["bbox_proportion_score"],
            "saliency_proportion_score": img.attrs["saliency_proportion_score"],
            "iou_score": img.attrs["iou_score"]
        }
        rows.append(row)

    df = pd.DataFrame(rows)
    df.to_json(out_f)
    

In [5]:
to_json_data(in_f, out_f)

In [6]:
df2 = pd.read_json(out_f)